# Finding an Object

So, we've spent the past two sections looking at pictures of a ladybug. Can we get the computer to recognize that there is a distinct subject of the photo?

Let's start by reloading our ladybug in.

In [66]:
import cv2
import numpy as np

img = cv2.imread("ladybug.png", cv2.IMREAD_GRAYSCALE)

## Detecting borders

So we're going to use a method called "edge detection." What is going on behind the scenes is fairly complicated and difficult to understand from a single workshop. So for now, understanding the output of the method is enough. If you are interested to learn more, here is a good place to start: [LINK]

We're going to be using a specific variation of edge detection called "Canny Edge Detection." Cv2 has a built-in function for this, `cv2.Canny`.

We're going to start by demonstrating how this works on the completely unedited image:

In [23]:
cv2.imshow("Color ladybug!", img)
cv2.waitKey(0)

edges_orig = cv2.Canny(img, 100, 200)

cv2.imshow("Temp", edges_orig)
cv2.waitKey(0)

-1

As you can see, the output shows an outline around most of the ladybug as well as some of the branch. 

`cv2.Canny` takes in 3 parameters: the image, the lower threshold, and the upper threshold. The thresholds refer to pixel intensities. You can try messing around with these and see how it affects the resulting image. (Without editing the image at all, it's unlikely this will end up yielding exceptional results.)

## Putting it together

We are now going to demonstrate the effects of some of the things we've previously learned on the accuracy of edge detection.

Let's start by adding a blur to our image and then retrying the edge detection.

In [29]:
blurred_img = cv2.GaussianBlur(img,(5,5),0)

edges_blurred = cv2.Canny(blurred_img, 100, 200)

cv2.imshow("Temp", edges_orig)
cv2.waitKey(0)
cv2.imshow("Temp", edges_blurred)
cv2.waitKey(0)

-1

Hopefully you can see that the lines are a little bit cleaner in the second image! We can try blurring it more to see if there is any added effect. Let's edit the kernel so the matrix is 15x15.

In [34]:
blurred_img = cv2.GaussianBlur(img,(15,15),0)

edges_blurred = cv2.Canny(blurred_img, 100, 200)

cv2.imshow("Temp", edges_orig)
cv2.waitKey(0)
cv2.imshow("Temp", edges_blurred)
cv2.waitKey(0)

-1

As you can see, the lines got much cleaner, but we also lost a decent portion of the ladybug. We can also see that there are large gaps in the edges. What can we do about this?

## Morphological Operations

So this is a big word that sounds much scarier than it actually is. There are 4 main operations that we'll be covering in this workshop:

1. Dilation: `cv2.dilate(image, kernel, iterations)`
2. Erosion: `cv2.erode(image, kernel, iterations)`
3. Opening: `cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)`
4. Closing: `cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)`

We'll show examples of each below.

### Dilation

Dilation is pretty much exactly what it sounds like: it "dilates" the foreground object of an image.

We have a good foreground with our image of the ladybug edges. Let's see what happens when we use cv2's erosion function! (We are using a comically large kernel in order to make the effects really obvious.)

In [55]:
kernel = np.ones((25,25),np.uint8)
dilation = cv2.dilate(edges_blurred,kernel,iterations = 1)

cv2.imshow("Temp", edges_blurred)
cv2.waitKey(0)
cv2.imshow("Eroded Borders", dilation)
cv2.waitKey(0)

-1

As you can see, the edges "dilated." They got much bigger! (You might be able to start seeing how this ties in: some of the gaps between sections are now gone!) While we did use a comically large kernel, this was actually effective in closing many of the gaps.

### Erosion

Erosion is also exactly what it sounds like: it "erodes" the foreground object of an image. Let's try eroding our new edges.

In [68]:
kernel = np.ones((11,11),np.uint8)
erosion = cv2.erode(dilation,kernel,iterations = 1)

cv2.imshow("Temp", dilation)
cv2.waitKey(0)
cv2.imshow("Eroded Borders", erosion)
cv2.waitKey(0)

-1

As you can see, we now have 2 main edges! Unfortunately the branch is still in the picture a little bit, but the ladybug is just one big white area!

Before we move on to labeling the original image, our final two morphological operations: opening and closing.

### Opening and Closing

Opening is another term for **erosion followed by dilation**. Closing is another term for **dilation followed by erosion.** The images below demonstrate how this can be useful for either removing noise from an image or filling in gaps.  As you can see, the main figure stays the same in both cases.

Opening:

![Opening](Opening.png)

Closing:

![Closing](Closing.png)

## Labeling the Image

So... we have our edges. How do we actually apply this to the original picture?

OpenCV has another built-in method to help you find the "contours" of an image. These are similar to the borders/boundaries. With our image already in black and white, it's extremely easy to find the contours using `cv2.findContours(image, retrieval method, approximation mode)`. For now, don't worry about what the methods are. If you are curious to look into it on your own, OpenCV's documentation has a section on retrieval modes and approximation modes that you can read.

We are then going to draw a rectangle around these contours using a built-in openCV method, `cv2.rectange`, which, as you can image, draws a rectangle.

In [72]:
contours, hierarchy = cv2.findContours(erosion, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

for contour in contours:
    # Find bounding rectangles
    x,y,w,h = cv2.boundingRect(contour)
    # Draw the rectangle
    cv2.rectangle(img,(x,y),(x+w,y+h),255,1)

cv2.imshow("Final", img)
cv2.waitKey(0)

-1

Overall, this is not bad! Using only the basic OpenCV methods, we can have the image recognize the central figure. We  have some smaller contours that, in the future, we will be able to get rid of fairly easily. We also have a bit of the branch still being labeled. 

As an exercise, you can try messing around with some of these steps to see if you can get rid of the branch!